### Packages

In [1]:
import os
import pickle
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from collections import Counter
from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier
import monpa
from monpa import utils
from collections import Counter
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

import datetime

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


### Data process

In [2]:
ptt_df = pd.read_csv("C:\\Users\\a0922\\Downloads\\bda2024_mid_dataset\\bda2024_202203-202402_討論數據_dcard.csv")
transaction_df = pd.read_csv("C:\\Users\\a0922\\Downloads\\bda2024_mid_dataset\\bda2024_微股力_個股交易數據-2年.csv")
with open('C:\\Users\\a0922\\Downloads\\stopwords_zh.txt', 'r') as file:
    stopwords = file.read().splitlines()
file.close()

C:\Users\a0922\AppData\Local\Temp\ipykernel_12060\444709794.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  transaction_df = pd.read_csv("C:\\Users\\a0922\\Downloads\\bda2024_mid_dataset\\bda2024_微股力_個股交易數據-2年.csv")


In [3]:
transaction_df['stock_symbol'] = transaction_df['stock_symbol'].astype(str)
dcard = ptt_df[ptt_df['title'].str.contains('聯發科') | ptt_df['content'].str.contains('聯發科')].reset_index(drop = True)


In [4]:
dcard

,id,forum,s_name,s_area_name,post_time,title,author,content,page_url,content_type
0,1646223888628_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-02 20:03:14.000,#標的 3/3 盤前功課,股施-交易日誌ami_0529/M,1. 今日開盤太樂觀，夜盤期貨一度跌快兩百點，結果開盤不過跌70點。期貨逆價差一直保持在50...,https://www.dcard.tw/f/stock/p/238253621,main
1,1646351877971_F0DCU,forum,校園聊天-Dcard,股票,2022-03-03 20:21:38.000,#請益 600以下台積電是不是上天賜予的禮物,國立中央大學/M,台積電的好，你知我知大家都知，\n是好公司好股票，但是也因為這樣，市場過度追捧下股價就會太貴...,https://www.dcard.tw/f/stock/p/238251620,reply
2,1646321103971_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-03 23:21:01.000,#標的 3/4 盤前功課,股施-交易日誌ami_0529/M,整體盤勢：66分（緩漲格局，一步一腳印）\n\n1. Fed主席鮑爾說升息一碼，如果通膨失控...,https://www.dcard.tw/f/stock/p/238265320,main
3,1646547422305_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-06 13:51:03.000,#標的 3/7 盤前功課,股施-交易日誌ami_0529/M,整體盤勢：43分（內外對抗，主流過熱，逢低佈局）\n\n1. 非農數據佳\n（更確立要緩步升...,https://www.dcard.tw/f/stock/p/238285624,main
4,1646599970728_F0DCU,forum,校園聊天-Dcard,股票,2022-03-06 23:52:25.000,六檔成長股，股利6元以上、殖利率7%以上,lunglailunglai/M,高殖利率然後又高成長的股票\n\n聯發科竟然沒人說\n\n預估今年會配70以上\n\n我覺得...,https://www.dcard.tw/f/stock/p/238267852,reply
...,...,...,...,...,...,...,...,...,...,...
1469,1709176681476_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-02-29 11:06:31.000,#分享 盤中焦點股,AT12接髮造型師at12ps38ps57/M,盤中焦點股：\n1.映泰(2399)：比特幣開年漲33％，今年拚破7萬美元，概念股狂歡，盤初...,https://www.dcard.tw/f/stock/p/254822868,main
1470,1709191273790_F0DCU,forum,校園聊天-Dcard,股票,2024-02-29 11:10:08.000,#分享 聯發科,鱷魚hao_0224_/M,6,https://www.dcard.tw/f/stock/p/254810282,reply
1471,1709238126746_F0DCU,forum,校園聊天-Dcard,股票,2024-02-29 15:40:11.000,#分享 聯發科,憲兵菁英特種是我pig400/M,https://megapx-assets.dcard.tw/images/5bdf19db...,https://www.dcard.tw/f/stock/p/254810282,reply
1472,1709238126823_F0DCU,forum,校園聊天-Dcard,股票,2024-02-29 21:28:42.000,#分享 聯發科,東海大學/M,讚讚,https://www.dcard.tw/f/stock/p/254810282,reply


### Post Cat.

In [5]:
def get_next_date(input_date, dates_list):
    index = sum(1 for d in dates_list if d < input_date)
    if index >= len(dates_list):
        return -1, -1
    else:
        return index, dates_list[index]

def get_answer(stock_name, input_date, n_days_after, threshold):
    dates_list = sorted(transaction_df.loc[(transaction_df['stock_name']==stock_name)]['date'])
    i, _ = get_next_date(input_date, dates_list)

    if i == -1 or i >= len(dates_list) or i + n_days_after >= len(dates_list):
        return 2
    
    open_date = dates_list[i]
    close_date = dates_list[i + n_days_after]
    
    
    open_price = transaction_df.loc[(transaction_df['date']==open_date) & (transaction_df['stock_name']==stock_name)]['open'].iloc[0]
    close_price = transaction_df.loc[(transaction_df['date']==close_date) & (transaction_df['stock_name']==stock_name)]['close'].iloc[0]
    price_change = (close_price - open_price) / open_price
    # print(f"{stock_name:5} | {price_change:5.2f} (Open: {open_date}, {open_price} | Close: {close_date}, {close_price})")

    if price_change >= threshold:
        return 1     # buy
    elif price_change <= -threshold:
        return -1    # sell
    else:
        return 0     # hold

In [12]:
dcard['stock_name'] = "聯發科"
dcard['category'] = dcard.apply(lambda r: get_answer(r['stock_name'], r['post_time'], 10, 0.02), axis = 1)

In [13]:
get_answer("聯發科", '2022-06-01', 10, 0.02)

-1

In [14]:
with open('dcard.pkl', 'wb') as file: 
    pickle.dump(dcard, file)

In [15]:
upSet = dcard[dcard['category'] == 1]
downSet = dcard[dcard['category'] == -1]

In [16]:
downSet

,id,forum,s_name,s_area_name,post_time,title,author,content,page_url,content_type,stock_name,category
0,1646223888628_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-02 20:03:14.000,#標的 3/3 盤前功課,股施-交易日誌ami_0529/M,1. 今日開盤太樂觀，夜盤期貨一度跌快兩百點，結果開盤不過跌70點。期貨逆價差一直保持在50...,https://www.dcard.tw/f/stock/p/238253621,main,聯發科,-1
1,1646351877971_F0DCU,forum,校園聊天-Dcard,股票,2022-03-03 20:21:38.000,#請益 600以下台積電是不是上天賜予的禮物,國立中央大學/M,台積電的好，你知我知大家都知，\n是好公司好股票，但是也因為這樣，市場過度追捧下股價就會太貴...,https://www.dcard.tw/f/stock/p/238251620,reply,聯發科,-1
2,1646321103971_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-03 23:21:01.000,#標的 3/4 盤前功課,股施-交易日誌ami_0529/M,整體盤勢：66分（緩漲格局，一步一腳印）\n\n1. Fed主席鮑爾說升息一碼，如果通膨失控...,https://www.dcard.tw/f/stock/p/238265320,main,聯發科,-1
11,1646910832279_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-10 18:55:45.000,#標的 3/11盤前功課,股施-交易日誌ami_0529/M,看文章前先了解自己屬於投資者/投機者\n發文問個股，請告訴我為什麼買這檔動機/背景/資金狀況...,https://www.dcard.tw/f/stock/p/238323041,main,聯發科,-1
19,1647435011218_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-16 20:40:25.000,#標的 3/17盤前功課,股施-交易日誌ami_0529/M,今日整體盤勢：19分\n明日盤勢預測：中性\n\n\n1. 外資現貨賣超150億元，期貨空單...,https://www.dcard.tw/f/stock/p/238371907,main,聯發科,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
1246,1704166018092_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-01-02 11:02:01.000,#新聞 2024年AI PC全面引爆！謝金河揭緯創等3檔接棒 電子5哥「它」最強,期貨女王dolag/F,AI PC是2024年投資市場最具共識的題材，謝金河表示，AI伺服器一哥緯創在90元附近盤整...,https://www.dcard.tw/f/stock/p/254306114,main,聯發科,-1
1247,1704168149710_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-01-02 11:29:31.000,#新聞 盤中焦點股,AT12接髮造型師at12ps38ps57/M,盤中焦點股：\n1.長榮(2603)：紅海緊張情勢未解，馬士基48小時暫停行經紅海，運費續漲...,https://www.dcard.tw/f/stock/p/254306291,main,聯發科,-1
1248,1704260908600_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-01-03 13:37:45.000,#分享 2024-01-02 台股籌碼看盤,紀律投資henrycheng23/M,"2024年新倉第一天加權指數下跌77.05點，收在17,853，萬八慶祝行情看來要再等等了。...",https://www.dcard.tw/f/stock/p/254316181,main,聯發科,-1
1249,1704287645529_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-01-03 20:55:19.000,#分享 #分享 大盤轉弱如何應對？以及後續如何觀察‼️ 避開風險的操作手法,簡單聊股 Ravenstockloging/M,"各位晚安,我是Raven.\n元旦個板貼文中提到大盤櫃買的關鍵位置 \n我們接著繼續觀察👀\...",https://www.dcard.tw/f/stock/p/254320000,main,聯發科,-1


### tf-idf

In [11]:
up_tf_counter=Counter()
up_df_counter=Counter()

for index, row in upSet.iterrows():
    query = row['title'] + ' ' + row['content']
    df_tmp = Counter()
    sentence_list = utils.short_sentence(query)

    for item in sentence_list:
      result_cut = monpa.cut(item) 
      for term in result_cut:
        term = term.strip()
        if (len(term) > 1):
          up_tf_counter[term] += 1 
          if(df_tmp[term] == 0): 
            df_tmp[term] = 1 
    up_df_counter += df_tmp

In [17]:
down_tf_counter=Counter()
down_df_counter=Counter()

for index, row in downSet.iterrows():
    query = row['title'] + ' ' + row['content']
    df_tmp = Counter()
    sentence_list = utils.short_sentence(query)

    for item in sentence_list:
      result_cut = monpa.cut(item) 
      for term in result_cut:
        term = term.strip()
        if (len(term) > 1):
          down_tf_counter[term] += 1 
          if(df_tmp[term] == 0): 
            df_tmp[term] = 1 
    down_df_counter += df_tmp

In [18]:
up_tf_idf = {}
d = len(upSet)
for term, freq in up_tf_counter.items():
    up_tf_idf[term] = freq * math.log(d / up_tf_counter[term])

In [19]:
down_tf_idf = {}
d = len(downSet)
for term, freq in down_tf_counter.items():
    down_tf_idf[term] = freq * math.log(d / down_tf_counter[term])

### Likelihood

In [17]:
frames = [upSet, downSet]
fullSet = pd.concat(frames).sort_values('post_time').reset_index()
fullSet

,index,id,forum,s_name,s_area_name,post_time,title,author,content,page_url,content_type,stock_name,category
0,0,1646223888628_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-02 20:03:14.000,#標的 3/3 盤前功課,股施-交易日誌ami_0529/M,1. 今日開盤太樂觀，夜盤期貨一度跌快兩百點，結果開盤不過跌70點。期貨逆價差一直保持在50...,https://www.dcard.tw/f/stock/p/238253621,main,聯發科,-1
1,1,1646351877971_F0DCU,forum,校園聊天-Dcard,股票,2022-03-03 20:21:38.000,#請益 600以下台積電是不是上天賜予的禮物,國立中央大學/M,台積電的好，你知我知大家都知，\n是好公司好股票，但是也因為這樣，市場過度追捧下股價就會太貴...,https://www.dcard.tw/f/stock/p/238251620,reply,聯發科,-1
2,2,1646321103971_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-03 23:21:01.000,#標的 3/4 盤前功課,股施-交易日誌ami_0529/M,整體盤勢：66分（緩漲格局，一步一腳印）\n\n1. Fed主席鮑爾說升息一碼，如果通膨失控...,https://www.dcard.tw/f/stock/p/238265320,main,聯發科,-1
3,5,1646659699708_F0DCU,forum,校園聊天-Dcard,股票,2022-03-07 13:19:42.000,#標的 3/7 盤前功課,國立雲林科技大學/M,想請問版主今天對聯發科的看法，謝謝您🙏,https://www.dcard.tw/f/stock/p/238285624,reply,聯發科,1
4,6,1646723296014_F0DCU,forum,校園聊天-Dcard,股票,2022-03-07 15:19:55.000,#請益 今天為什麼會崩成這樣?,國立高雄應用科技大學/M,利空事實就擺在那邊啊 \n烏俄戰爭的確是短期利空 但你有沒有想過背後的經濟危機不是隨便跌個4...,https://www.dcard.tw/f/stock/p/238294503,reply,聯發科,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,1335,1706240338414_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-01-26 10:53:40.000,#請益 台股籌碼面的影響究竟有多大？要從哪裡開始學習籌碼分析？,國立臺灣大學/M,大家好，我是去年剛畢業的新鮮人，股齡大概一年左右而已，剛好父親之前有給我一百萬左右去練習投資...,https://www.dcard.tw/f/stock/p/254523838,main,聯發科,1
799,1336,1706286879139_F0DCU,forum,校園聊天-Dcard,股票,2024-01-26 18:34:51.000,#分享 你今天linepay了嗎？,中華醫事科技大學/M,哇靠！比聯發科還賺錢的公司？,https://www.dcard.tw/f/stock/p/254524307,reply,聯發科,1
800,1337,1706589387337_F0DCU,forum,校園聊天-Dcard,股票,2024-01-30 11:35:53.000,#分享 終於把虧損回補,國立清華大學/F,今天走跌了，應該換其他半導體漲了，聯發科，聯電之類，算是投機財還是要小心點。,https://www.dcard.tw/f/stock/p/254552476,reply,聯發科,1
801,1338,1706591072217_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-01-30 12:11:00.000,#新聞 盤中焦點股,AT12接髮造型師at12ps38ps57/M,《台北股市》盤中焦點股：英濟、雙鴻、奇鋐、健策、蔚華科、應廣\n2024年1月30日 週二 ...,https://www.dcard.tw/f/stock/p/254556265,main,聯發科,1


In [18]:
import re
def clearSentence(sentence):
    return re.sub(r'[^\u4e00-\u9fa5]+', '', sentence)

In [22]:
def cal_dict_docs(docsFrame):
  df_counter=Counter()

  for index, row in docsFrame.iterrows():
      query = row['title'] + ' ' + row['content']
      df_tmp = Counter()
      sentence_list = utils.short_sentence(query)

      for item in sentence_list:
        item = clearSentence(item)
        result_cut = monpa.cut(item) 
        for term in result_cut:
          term = term.strip()
          if (len(term) > 1):
            if(df_tmp[term] == 0): 
              df_tmp[term] = 1 
      df_counter += df_tmp
  return df_counter

In [23]:
def cal_term_docs(docsRow):
  tf_counter=Counter()

  for index, row in docsRow.iterrows():
      query = row['title'] + ' ' + row['content']
      sentence_list = utils.short_sentence(query)
      
      for item in sentence_list:
        item = clearSentence(item)
        result_cut = monpa.cut(item) 
        for term in result_cut:
          term = term.strip()
          if (len(term) > 1):
            tf_counter[term]+=1
  return tf_counter

In [25]:
def likeliratio_selection(docsFrame, trainCateg=[], nFeatrues=500):
    tfDocsList = {}
    vocabDict = cal_dict_docs(docsFrame)
    ScoreList = {}
        
    for i in range(len(docsFrame)):
        tfDocsList[i] = cal_term_docs(docsFrame.loc[[i]])
        
    for term in vocabDict:
        Score = 0
        for categ in trainCateg:
            real = np.zeros((2,2))
            for index, row in docsFrame.iterrows():
                if (row['category'] == trainCateg[categ]):
                    if (term in tfDocsList[index]):
                        real[0][0] += 1
                    else:
                        real[0][1] += 1
                else:
                    if (term in tfDocsList[index]):
                        real[1][0] += 1
                    else:
                        real[1][1] += 1
            sumTotal = real.sum()
            pt = (real[0][0] + real[1][0]) / sumTotal
            p1 = real[0][0] / (real[0][0] + real[0][1])
            p2 = real[1][0] / (real[1][0] + real[1][1])
            lambdaVal = ((pt ** real[0][0]) * ((1 - pt) ** real[0][1]) * (pt ** real[1][0]) * ((1 - pt) ** real[1][1])) / ((p1 ** real[0][0]) * ((1 - p1) ** real[0][1]) * (p2 ** real[1][0]) * ((1 - p2) ** real[1][1]))
            Score += -2 * math.log(lambdaVal)
        ScoreList[term] = Score
    selected = sorted(ScoreList, key=ScoreList.get, reverse=True)[:nFeatrues]
    totalList = sorted(ScoreList, key=ScoreList.get, reverse=True)
    return selected, totalList

In [26]:
result_keywords, total_keywords = likeliratio_selection(fullSet, trainCateg = [1, -1], nFeatrues = 2000)

In [27]:
with open('result_keywords.pkl', 'wb') as file: 
    pickle.dump(result_keywords, file)

In [28]:
with open('total_keywords.pkl', 'wb') as file: 
    pickle.dump(total_keywords, file)

In [136]:
tmp = result_keywords
result_keywords = total_keywords[:1000]
result_keywords 

['跌停',
 '益聯發科',
 '明天',
 '高台',
 '這麼',
 '標的',
 '適合',
 '除息',
 '貼息',
 '龍頭',
 '入袋',
 '大摩',
 '分批',
 '半導體',
 '高台積電',
 '積電',
 '分享',
 '烙賽',
 '追高',
 '挺聯',
 '嚴重',
 '那麼',
 '小白',
 '怎麼',
 '股癌',
 '畢業',
 '入手',
 '小弟',
 '下車',
 '買入',
 '回歸',
 '美系',
 '乾貨',
 '解套',
 '攤平',
 '聯發科',
 '背景',
 '一樣',
 '持有',
 '戰略',
 '什麼',
 '過度',
 '動機',
 '繼續',
 '股息',
 '技術',
 '腰斬',
 '啟動',
 '私訊',
 '年減',
 '建雄',
 '雄才',
 '使用',
 '大於',
 '口量',
 '水餃股',
 '第三代',
 '一段',
 '機會',
 '隨時',
 '同學',
 '現貨',
 '朋友',
 '有望',
 '順利',
 '耐心',
 '盤前',
 '兩成',
 '懷疑',
 '八成',
 '深入',
 '代工車',
 '無論',
 '相反',
 '例子',
 '正確',
 '精星',
 '祥碩',
 '公告',
 '創意',
 '功課',
 '準備',
 '分析',
 '緯創',
 '收入',
 '美國聯準會',
 '開出',
 '當日',
 '取得',
 '引發',
 '不佳',
 '量產',
 '競爭',
 '和碩',
 '零組件',
 '政治',
 '改善',
 '給予',
 '證明',
 '一舉',
 '話題',
 '百億',
 '矽統',
 '投機人',
 '思維',
 '告訴',
 '開低',
 '股價',
 '夜盤',
 '祝福',
 '轉賣',
 '緩步',
 '找尋',
 '合晶',
 '朝向',
 '依循',
 '漲到',
 '盡量',
 '空軍',
 '到來',
 '知識',
 '加油',
 '半順德',
 '墊高',
 '幫忙',
 '回補',
 '千金',
 '投資者',
 '上來',
 '伺服器',
 '想要',
 '樂觀',
 '期貨',
 '買來',
 '合適',
 '逢低',
 '政府',
 '關係',
 '高價股',
 '汽車',
 '

### Sets prepare

In [44]:
with open('result_keywords.pkl', 'rb') as f:
    result_keywords = pickle.load(f)

In [19]:
fullSet['post_time'] = pd.to_datetime(fullSet['post_time']).dt.date

In [20]:
fullSet

,index,id,forum,s_name,s_area_name,post_time,title,author,content,page_url,content_type,stock_name,category
0,0,1646223888628_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-02,#標的 3/3 盤前功課,股施-交易日誌ami_0529/M,1. 今日開盤太樂觀，夜盤期貨一度跌快兩百點，結果開盤不過跌70點。期貨逆價差一直保持在50...,https://www.dcard.tw/f/stock/p/238253621,main,聯發科,-1
1,1,1646351877971_F0DCU,forum,校園聊天-Dcard,股票,2022-03-03,#請益 600以下台積電是不是上天賜予的禮物,國立中央大學/M,台積電的好，你知我知大家都知，\n是好公司好股票，但是也因為這樣，市場過度追捧下股價就會太貴...,https://www.dcard.tw/f/stock/p/238251620,reply,聯發科,-1
2,2,1646321103971_F02_dcard,forum,校園聊天-Dcard,股票,2022-03-03,#標的 3/4 盤前功課,股施-交易日誌ami_0529/M,整體盤勢：66分（緩漲格局，一步一腳印）\n\n1. Fed主席鮑爾說升息一碼，如果通膨失控...,https://www.dcard.tw/f/stock/p/238265320,main,聯發科,-1
3,5,1646659699708_F0DCU,forum,校園聊天-Dcard,股票,2022-03-07,#標的 3/7 盤前功課,國立雲林科技大學/M,想請問版主今天對聯發科的看法，謝謝您🙏,https://www.dcard.tw/f/stock/p/238285624,reply,聯發科,1
4,6,1646723296014_F0DCU,forum,校園聊天-Dcard,股票,2022-03-07,#請益 今天為什麼會崩成這樣?,國立高雄應用科技大學/M,利空事實就擺在那邊啊 \n烏俄戰爭的確是短期利空 但你有沒有想過背後的經濟危機不是隨便跌個4...,https://www.dcard.tw/f/stock/p/238294503,reply,聯發科,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
798,1335,1706240338414_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-01-26,#請益 台股籌碼面的影響究竟有多大？要從哪裡開始學習籌碼分析？,國立臺灣大學/M,大家好，我是去年剛畢業的新鮮人，股齡大概一年左右而已，剛好父親之前有給我一百萬左右去練習投資...,https://www.dcard.tw/f/stock/p/254523838,main,聯發科,1
799,1336,1706286879139_F0DCU,forum,校園聊天-Dcard,股票,2024-01-26,#分享 你今天linepay了嗎？,中華醫事科技大學/M,哇靠！比聯發科還賺錢的公司？,https://www.dcard.tw/f/stock/p/254524307,reply,聯發科,1
800,1337,1706589387337_F0DCU,forum,校園聊天-Dcard,股票,2024-01-30,#分享 終於把虧損回補,國立清華大學/F,今天走跌了，應該換其他半導體漲了，聯發科，聯電之類，算是投機財還是要小心點。,https://www.dcard.tw/f/stock/p/254552476,reply,聯發科,1
801,1338,1706591072217_F02_dcard_C,forum,校園聊天-Dcard,股票,2024-01-30,#新聞 盤中焦點股,AT12接髮造型師at12ps38ps57/M,《台北股市》盤中焦點股：英濟、雙鴻、奇鋐、健策、蔚華科、應廣\n2024年1月30日 週二 ...,https://www.dcard.tw/f/stock/p/254556265,main,聯發科,1


In [63]:
train_startDate1 = datetime.date(2022,3,1)
train_endDate1 = datetime.date(2023,6,30)
trainSet = fullSet[fullSet['post_time'].between(train_startDate1,train_endDate1)]

len(trainSet[trainSet['category'] == -1])

396

In [22]:
loop = tqdm(enumerate(trainSet.iterrows(), 1), total=len(trainSet), leave=True)

train_tokenStr_list1 = []
for index,i in loop:       
    #以下套用斷詞小範例
    query=i[1]["title"]+' '+i[1]['content']
    sentence_list = utils.short_sentence(query) #斷句
    tokenStr = str()
    for sentence in sentence_list:
        sentence = clearSentence(sentence)
        tokens = monpa.cut(sentence) #斷詞
        tokenStr += ' '.join(tokens)
    train_tokenStr_list1.append(tokenStr)

100%|██████████| 557/557 [00:27<00:00, 20.31it/s]


In [65]:
test_startDate1 = datetime.date(2023,7,1)
test_endDate1 = datetime.date(2024,2,29)
testSet = fullSet[fullSet['post_time'].between(test_startDate1,test_endDate1)]

len(testSet[testSet['category'] == 1])

188

In [24]:
loop = tqdm(enumerate(testSet.iterrows(), 1), total=len(testSet), leave=True)

test_tokenStr_list1 = []
for index,i in loop:       
    #以下套用斷詞小範例
    query=i[1]["title"]+' '+i[1]['content']
    sentence_list = utils.short_sentence(query) #斷句
    tokenStr = str()
    for sentence in sentence_list:
        sentence = clearSentence(sentence)
        tokens = monpa.cut(sentence) #斷詞
        tokenStr += ' '.join(tokens)
    test_tokenStr_list1.append(tokenStr)

100%|██████████| 246/246 [00:12<00:00, 19.89it/s]


In [90]:
vectorizer = TfidfVectorizer(stop_words=stopwords)
X_train1 = vectorizer.fit_transform(train_tokenStr_list1)
X_train1 = pd.DataFrame(X_train1.toarray(),columns=vectorizer.get_feature_names_out())
X_train1

c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


,一下來,一下另外,一下車,一併,一兩,一千,一半,一半且,一口氣,一咪咪,...,點錢,點陣圖,黯淡,鼎立積致,鼓吹成,鼓鼓,齊漲,龍頭,龍頭廠,龐大
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
y_train1 = trainSet['category']

chi2_selector = SelectKBest(chi2, k = 2000)
chi2_selector.fit(X_train1, y_train1)
kbest_vocabs = X_train1.columns[chi2_selector.get_support()]
X_train1 = X_train1[kbest_vocabs]
X_train1N = X_train1 


In [92]:
y_test1 = testSet['category']

vectorizer = TfidfVectorizer(stop_words=stopwords)
X_test1 = vectorizer.fit_transform(test_tokenStr_list1)
X_test1 = pd.DataFrame(X_test1.toarray(),columns=vectorizer.get_feature_names_out())
X_test1 = X_test1.reindex(kbest_vocabs, axis=1, fill_value=0)



X_test1

c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(


,一下另外,一併,一兩,一千,一半,一年,一年期,一律,一早,一月,...,鴻海聯電,鴻海鴻準台泥,麻煩,黃董,黃金,默默,點元,點數,鼎立積致,鼓鼓
0,0,0.0,0,0,0.10728,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0
1,0,0.0,0,0,0.00000,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0
2,0,0.0,0,0,0.00000,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0
3,0,0.0,0,0,0.00000,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0
4,0,0.0,0,0,0.00000,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,0,0.0,0,0,0.00000,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0
242,0,0.0,0,0,0.00000,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0
243,0,0.0,0,0,0.00000,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0
244,0,0.0,0,0,0.00000,0,0,0,0.0,0,...,0,0,0.0,0,0.0,0.0,0,0,0,0


In [110]:
#clf = BernoulliNB(alpha=1.0)
#clf = MLPClassifier(hidden_layer_sizes=(500, 50), max_iter=300)
#clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1, max_depth=7, random_state=0)
clf = DecisionTreeClassifier(max_depth=10, min_samples_split=2, min_samples_leaf=1, max_features=None)
#clf = KNeighborsClassifier(n_neighbors = 3, weights = 'distance',algorithm = 'auto')
clf.fit(X_train1, y_train1)
clf.score(X_train1, y_train1)

0.8330341113105925

In [114]:
test_label = testSet.groupby(['post_time','category']).count().sort_values('category', ascending = False).sort_index(level=[0], sort_remaining=False).groupby(level=0).head(1).reset_index()['category']
testSet['predict_label'] = clf.predict(X_test1)


predict_label = testSet.groupby(['post_time','predict_label']).count().sort_values('predict_label', ascending = False).sort_index(level=[0], sort_remaining=False).groupby(level=0).head(1).reset_index()['predict_label']
print(predict_label)

from sklearn.metrics import accuracy_score
accuracy_score(test_label , predict_label)
print('2023.7 ~ 2024.2預測準確率:', accuracy_score(test_label, predict_label))

0    -1
1    -1
2     1
3     1
4     1
     ..
72    1
73    1
74    1
75    1
76    1
Name: predict_label, Length: 77, dtype: int64
2023.7 ~ 2024.2預測準確率: 0.6493506493506493


C:\Users\a0922\AppData\Local\Temp\ipykernel_12060\369224264.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testSet['predict_label'] = clf.predict(X_test1)


In [112]:
cm = confusion_matrix(test_label, predict_label )
cm

array([[ 2, 19],
       [ 8, 48]], dtype=int64)

BackTest


In [115]:
start_date = datetime.date(2022, 6, 1)
end_date = datetime.date(2024, 1, 31)
current_date = start_date
Score = []
result = []
Date = []
while current_date <= end_date - datetime.timedelta(days=26):
    train_startDate = current_date - datetime.timedelta(days=90)
    train_endDate = current_date - datetime.timedelta(days=1)
    print(f"Train: {train_startDate} ~ {train_endDate}")
    test_startDate = current_date
    if current_date.strftime("%m") in ['01','03','05','07','08','10','12']:
        test_endDate = test_startDate + datetime.timedelta(days=30)
    elif current_date.strftime("%m") in ['04','06','09','11']:
        test_endDate = test_startDate + datetime.timedelta(days=29)
    else:
        test_endDate = test_startDate + datetime.timedelta(days=27)
    print(f"Test: {test_startDate} ~ {test_endDate}")

    #train
    train_tokenStr_list = []
    date_list = []
    for i in list(fullSet[fullSet['post_time'].between(train_startDate, train_endDate)].index):
        try:
            sentence_list = utils.short_sentence(fullSet['content'][i])
            tokenStr = str()
            for sentence in sentence_list:
                sentence = clearSentence(sentence)
                tokens = monpa.cut(sentence)
                tokenStr += ' '.join(tokens)
            train_tokenStr_list.append(tokenStr)
        except:
            #print error
            #print(fullSet['content'][i])
            train_tokenStr_list.append('')

    print(train_tokenStr_list)
    vectorizer = TfidfVectorizer(stop_words=stopwords)
    y_train = fullSet[fullSet['post_time'].between(train_startDate, train_endDate)]['category']

    X_train = vectorizer.fit_transform(train_tokenStr_list)
    X_train = pd.DataFrame(X_train.toarray(),columns=vectorizer.get_feature_names_out())
    X_train[[col for col in X_train.columns if col in result_keywords]]

    X_trainN = pd.DataFrame(index=range(0,len(X_train)), columns=result_keywords).fillna(0)
    X_trainN[[col for col in X_train.columns if col in result_keywords]] = X_train[[col for col in X_train.columns if col in result_keywords]]

    # #chi2 feature selection
    # chi2_selector = SelectKBest(chi2, k = 500)
    # chi2_selector.fit(X_train, y_train)
    # kbest_vocabs = X_train.columns[chi2_selector.get_support()]
    # X_trainN = X_train[kbest_vocabs]


    #test
    test_tokenStr_list = []

    for i in list(fullSet[fullSet['post_time'].between(test_startDate, test_endDate)].index):
        date_list.append(fullSet['post_time'][i])
        try:
            sentence_list = utils.short_sentence(fullSet['content'][i])
            tokenStr = str()
            for sentence in sentence_list:
                sentence = clearSentence(sentence)
                tokens = monpa.cut(sentence)
                tokenStr += ' '.join(tokens)
            test_tokenStr_list.append(tokenStr)
        except:
            test_tokenStr_list.append('')
    
    y_test = fullSet[fullSet['post_time'].between(test_startDate, test_endDate)]['category']

    vectorizer = TfidfVectorizer(stop_words=stopwords)
    X_test = vectorizer.fit_transform(test_tokenStr_list)
    X_test = pd.DataFrame(X_test.toarray(),columns=vectorizer.get_feature_names_out())

    X_testN = pd.DataFrame(index=range(0,len(X_test)), columns=result_keywords).fillna(0)
    X_testN[[col for col in X_test.columns if col in result_keywords]] = X_test[[col for col in X_test.columns if col in result_keywords]]

    # vectorizer = TfidfVectorizer(stop_words=stopwords)
    # X_testN = X_test.reindex(kbest_vocabs, axis=1, fill_value=0)

    #model
    clf = DecisionTreeClassifier(max_depth=10, min_samples_split=2, min_samples_leaf=1, max_features=None)
    clf.fit(X_trainN, y_train)
    
    Score.append(clf.score(X_testN, y_test))
    print(Score)
    Predict = clf.predict(X_testN)
    print(Predict)
    result.append(Predict)
    Date.append(date_list)

    print(current_date.strftime("%Y/%m/%d"))
    if current_date.strftime("%m") in ['01','03','05','07','08','10','12']:
        current_date = current_date + datetime.timedelta(days=31)
    elif current_date.strftime("%m") in ['04','06','09','11']:
        current_date = current_date + datetime.timedelta(days=30)
    else:
        current_date = current_date + datetime.timedelta(days=28)
    
    #for i in list(fullSet[fullSet['post_time'].between(test_startDate, test_endDate)].index):
        #date_lst.append(fullSet['post_time'][i])

    #result.update(dict(zip(date_lst, Predict)))

Train: 2022-03-03 ~ 2022-05-31
Test: 2022-06-01 ~ 2022-06-30
['台積電 的 好 你 知 我 知 大家 都 知 是 好 公司 好 股票 但是 也 因為 這樣 市場 過度 追捧 下 股價 就 會 太 貴 可能 年後 它 的 合理價 才 會 是 年後 合理價 才 是 所以 我 買 聯 發科 聯詠 群聯 瑞昱', '整體 盤勢 分 緩漲 格局 一 步 一 腳印 主席 鮑爾 說 升息 一 碼 如果 通膨 失 控會 考慮 一 次 升息 兩 碼 或 增加 升息 頻率縮表 這 個 月 不會 討論昨天 說 過 了 但 很 重要 再 說 一 遍 期貨 選擇權 皆 中性 偏 樂觀 看待 即便 美股 大漲 短線 上 很 難 突破 跟 兩 大 關卡 外資 還 未 回頭 轉買 期貨 選擇權 皆 顯示 不會 讓 指數 輕易 上揚 因此 個人 認為 短 時 都 不會 拉抬 權值股 例如 台積電 買 權值股 的 投資 朋友 可能 要 有點 耐心 逢 請 適量 獲利 出場 幫 自己 加菜 這樣 低點 才 有 籌碼 可以 買 相對 的 以下 有 強力 支撐即便 有 大利 空 短 時間 守 得 住假設 繼續 壓 台積電 那 中小型 個股 可能 還 有 空間 辛苦 護國 神山 了 投資人 思維 再次 強調 有 漲 到 萬八 短線 上 一律 建議 先 部份 獲利 要 賣 紅 才 有 錢 買 黑 投資者 標 的 建議 請 自行 研究 個股 完整 基本面 再 進場 鴻海 台達電 台郡 環球晶 皆 處 相對 低點 適合 買來 佈局 中 長線 聯詠合晶 至 上 佳世達 華碩聯發科 發動 中 或 在 高檔 中 目前 被 低估 的 投機人 思維 一樣 以 選股 為主多空 都 好 做 創惟智原 建議 當 沖這 兩 檔 還 沒 買 的 就 不 建議 同學 們 進場 抱 隔天 了 創 惟盤 中 一直 鎖不住 觀望 力道 已經 大於 買盤 了 智原 突破 昨 高 馬上 打下來 顯示 追價 買盤 也 不足 雖然 我 還 有 持股 但 平心 而 論 真的 距離 修正 不遠 鋼鐵 與 我 無緣 我 決定 離開 傷心地海 可以 持續 緩漲 觀察 另外 兩 檔 會 不會 補漲 飛機 今天 拉上 去 我 反而 不會 進場 我 個人 喜愛 碰 月線 洗籌碼 再上 持續 觀望 假設 

c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(
c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②

[0.8928571428571429]
[-1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1  1 -1 -1  1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  1 -1  1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1
 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
2022/06/01
Train: 2022-04-02 ~ 2022-06-30
Test: 2022-07-01 ~ 2022-07-31
['回到 疫情 前 再 叫 我', '多久 前 的 新聞 了 不要 安慰 自己 了', '技術面 上來 看 完全 沒 看到 止穩 的 跡象 然後 技術面 又 大於 消息面 所以 明天 再 向 下 一 根 挑戰 低點', '這 是 幾 個 禮拜 前 早 就 在 討論 的 事 了 欸', '請 看看 台積電 財報 都 出來 還 不 是 向下', '比 大盤 還 弱勢 的 股票 都 破 年線 了 年線 半年線 都 下彎 大 股東 跑光 了 我 看', '覆巢 之下 無 完卵 基本面 不如 一 碗 泡麵 升息 資金 回 美國 了 還 跟 你 買 台股 幹 嘛', '認同 股價 會 從 一路 噴上去 是 因為 疫情 讓 需求量 大增 慢慢 解封 市場 對 這 塊 需求 還是 有 但 沒有 疫情 下 大量 需求 再來 就 是 美國 升息 收回 大量 熱錢 自然 市場 資金 不會 投 這麼多 不過 說來 原因 百百 種 趕快 止損 留 著 銀彈 等 下 一 波 實在', '已經 刪除 的 內容 就 像 一樣 錯過 是 無法 再 相見 的', '台積電 都 沒 噴 了 是 要 噴 什麼 挖溝', '我 也 噴 了', '停損 不然 就 攤平 啊 分批 資金 跌到 哪 攤 到 哪 如果 只 會 滿倉 到時候 沒 錢 又 不 停損 

c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(
c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②

[0.8928571428571429, 0.17857142857142858]
[-1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1  1 -1  1]
2022/07/01
Train: 2022-05-03 ~ 2022-07-31
Test: 2022-08-01 ~ 2022-08-31
['本人 是 三月 才 剛 開戶 的 新手 去年 大學 畢業 第一 份 工作 快 一年 三月 時 看到 聯發科 股價 在 開始 跌 的 時候 就 進場 了 連 買 幾 天 沒 錢 了 結果 還 繼續 跌慘 請 問版 上 的 大神 我 應該 要 止損 嗎 還是 可以 這 個 月 繼續 加碼 這 筆 錢 不 大 只 有 幾萬 但是 我 拿來 投資 的 第一 筆錢 難免 會 有點 心疼 其實 當作 存股 繼續 放 著 也 可以 我 就 怕 它 一路 往下 到腰斬', '台積電 是 大盤 我 覺得 還好 聯發科 比較 危險 怕 跌 不要 買 高價股 也 不要 買 股 本 小 的 聯發科 要 的 話 可以 小攤 一點 也許 還 會 跌 但 聯發科 好像 也 算 權值股 我 看 他 點數 排名 還 滿 前面 的 是 因為 他 貴 嗎', '台積電 全球 第一 晶圓 代工廠 聯發科 台灣 第一 設計 公司 這 兩 間 如果 要 停損 的 話 建議 你 不要 碰 電子股 了 買 中華電 就 好', '我 是 覺得 你 能 繼續 放 著 未來 還 能 驕傲 的 說 我 有 買 台積電 跟 聯發科', '聯發科 就 放 著 別 加碼 了 至少 有 每 年 股利', '起 外資 多 空單 股 指期貨 多單 平倉 空單 增 累積 多單 空單 股票 期貨 多單 平倉單 空單 增單 累積 多單 空單 選擇權 賣權 買權 台積電聯電 聯發科 來 個 悶殺 聯電 漲停 也 可以 啦 本 日 聯電 空多', '會 用 手機 市場 狀況 來 評斷 聯發科 的 只 能 說 還是 不要 給 建議 的 好', '其實 不管 你 要 投聯 發科 還是 台積電 還是 低 一點 的 股票 你 每 個 月 就是 投萬 啊 根本 跟 股價 高低 無 所謂 你 都 是 買 零股 純粹 比較 快 湊到 一 張 而已', '認真

...

[0.8928571428571429, 0.17857142857142858, 0.75, 0.9285714285714286, 0.4, 0.2, 0.5625, 0.0, 0.8333333333333334, 0.625, 0.10204081632653061, 0.1, 0.5882352941176471, 0.75, 0.391304347826087, 0.85, 0.96, 0.8947368421052632]
[-1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1]
2023/11/01
Train: 2023-09-02 ~ 2023-11-30
Test: 2023-12-01 ~ 2023-12-31
['前 幾 天 本來 想要 買 聯發科 的 那 時候 本來 我 沒有 下手 最近 他 漲 到 我 又 覺得 太 高 所以 我 就 買 當時 我 覺得 買 在 覺得 好像 很 高 了 但是 我 又 看 了 一下 大盤 最近 的 市加權 指數 那時候 還 算 在 近期 低檔 季均線 跟 半年線 附近 所以 我 就 果 斷 買 了 今天 看到 漲 直接 上班 高潮', '想 問 小熊 聯發科 的 目標價 謝謝', '請問 小熊 華碩 均價 微星 均價 聯發科 均價 敦揚 科 均價 華航 均價 聯詠 均價 台 勝科 均價 神達 均價 環球 晶愛普龍德造 船 細 創 有 哪些 還 可以 抱 的 嗎', '大家 好 我 是 建雄 分析師 美股 昨日 因 安謀 上市 而 被 帶動 大漲 台股 昨日 就 已 提前 反應 上漲 目前 台股 上檔 遇到 季線 反壓 所以 短線 會 進入 震盪 不過 由於 月線 已 翻揚 往 上 上漲 格局 不 看 壓力 看 支撐 所以 只要 拉回 不 破 月線 行情 仍 是 偏多 操作 為主 後勢 應 是 指數 震盪 後 再 突破 季線 反壓 高價股 擁有 籌碼 安定 優勢 而 領先 上漲 譜 瑞群聯智邦 川湖詠矽 創 創意 聯陽聯詠聯發科 都 是 本土 法人 認養 的 標的 股價 走勢 偏向 溫和 慢慢 往 上漲 可 避免 當 沖客 短線客 帽客

c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(
c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②

[0.8928571428571429, 0.17857142857142858, 0.75, 0.9285714285714286, 0.4, 0.2, 0.5625, 0.0, 0.8333333333333334, 0.625, 0.10204081632653061, 0.1, 0.5882352941176471, 0.75, 0.391304347826087, 0.85, 0.96, 0.8947368421052632, 0.20408163265306123]
[ 1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1]
2023/12/01
Train: 2023-10-03 ~ 2023-12-31
Test: 2024-01-01 ~ 2024-01-31
['兄弟 們 扛不住 了 部隊 生活 分享 與 心理 醫生 的 聊天 分享 近期 市場 話題 聯發科 反彈 後 又 被 美股 帶 下去 基本面 來 看 還是 偏 高估 堅持 自己 選擇 的 策略不要 人 云 亦 云 達發 整體 展望 很 好 人 才 也 多 但 問題 就 是 太 貴 手機 部分 庫存 已 降到 極 低 只是 需求 尚未 回歸 對 發哥 高通 而言 手機 就 當 保底 主要 關注 其他 業務 發哥 年會 與 合作 車機 電競 可 視為 一 大利多 各 家 公司 估值 應 要 拆解 各 業務 並 給 個別 估值 再 加 總 近期 市場 話題 心態 近期 討論 的 大 公司 漲幅 排除 後 其他 根本 沒 漲 雖然 值得 注意但 市值 排列 最 大 的 當然 影響 也 最 大 所以 過去 發生 過 的 市況 不 代表 現在 市況 會 重演 且 現在 利率 過 高 許多 新創 公司 都 死 得 差不多 了 導致 市場 共識 都 把 資金 拿去 買 有 題材 且 大型 的 公司 如果 你 相信 這 狀況 不 合理 那 你 就 應該 空進去 然後 畢業 尤其 你 是 市場仔 卻 沒 追到 這些 題材 那 更 應該 檢討 自己 不 過 的 高地 是 真

c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  warnings.warn(
c:\Users\a0922\miniconda3\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②

[0.8928571428571429, 0.17857142857142858, 0.75, 0.9285714285714286, 0.4, 0.2, 0.5625, 0.0, 0.8333333333333334, 0.625, 0.10204081632653061, 0.1, 0.5882352941176471, 0.75, 0.391304347826087, 0.85, 0.96, 0.8947368421052632, 0.20408163265306123, 0.8636363636363636]
[ 1 -1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1
  1  1  1  1  1  1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1]
2024/01/01


In [116]:
def daily_predictions(time_list, prediction_list):
    daily_results = {}
    
    for time, prediction in zip(time_list, prediction_list):
        # 將日期轉換為每日日期
        daily_date = time
        
        # 如果這一天還沒有預測結果的列表，就建立一個新的列表
        if daily_date not in daily_results:
            #print(daily_date)
            daily_results[daily_date] = []
        
        # 將預測結果加入到該日期的列表中
        daily_results[daily_date].append(prediction)
    
    # 進行投票
    daily_votes = {}
    for date, predictions in daily_results.items():
        # 計算每個日期的投票結果
        positive_count = sum(1 for result in predictions if result == 1)
        negative_count = sum(1 for result in predictions if result == -1)
        
        # 決定每日的投票結果
        if positive_count > negative_count:
            daily_votes[date] = 1
        elif positive_count < negative_count:
            daily_votes[date] = -1
        else:
            # 如果正負票數相等，你可以採取一個預設的決策，比如0或者None
            daily_votes[date] = 0
    
    return daily_votes

In [117]:
#flat Date
Date = [item for sublist in Date for item in sublist]
Date

[datetime.date(2022, 6, 1),
 datetime.date(2022, 6, 1),
 datetime.date(2022, 6, 1),
 datetime.date(2022, 6, 1),
 datetime.date(2022, 6, 2),
 datetime.date(2022, 6, 6),
 datetime.date(2022, 6, 7),
 datetime.date(2022, 6, 8),
 datetime.date(2022, 6, 9),
 datetime.date(2022, 6, 10),
 datetime.date(2022, 6, 15),
 datetime.date(2022, 6, 17),
 datetime.date(2022, 6, 18),
 datetime.date(2022, 6, 20),
 datetime.date(2022, 6, 20),
 datetime.date(2022, 6, 20),
 datetime.date(2022, 6, 20),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 22),
 datetime.date(2022, 6, 23),
 datetime.date(2022, 6, 23),
 datetime.date(2022, 6, 23),
 datetime.date(2022, 6, 23),
 datetime.date(2022, 6,

In [118]:

result = [item for sublist in result for item in sublist]
result

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -1,
 1,
 1,
 1,
 1,
 1,
 -1,
 -1,
 -1,
 -1,
 

In [119]:
#flat the result
daily_votes = daily_predictions(Date, result)
daily_votes

{datetime.date(2022, 6, 1): -1,
 datetime.date(2022, 6, 2): -1,
 datetime.date(2022, 6, 6): -1,
 datetime.date(2022, 6, 7): 1,
 datetime.date(2022, 6, 8): -1,
 datetime.date(2022, 6, 9): -1,
 datetime.date(2022, 6, 10): -1,
 datetime.date(2022, 6, 15): -1,
 datetime.date(2022, 6, 17): -1,
 datetime.date(2022, 6, 18): -1,
 datetime.date(2022, 6, 20): 0,
 datetime.date(2022, 6, 22): -1,
 datetime.date(2022, 6, 23): -1,
 datetime.date(2022, 6, 24): -1,
 datetime.date(2022, 6, 25): -1,
 datetime.date(2022, 6, 27): -1,
 datetime.date(2022, 7, 1): -1,
 datetime.date(2022, 7, 2): -1,
 datetime.date(2022, 7, 3): -1,
 datetime.date(2022, 7, 4): -1,
 datetime.date(2022, 7, 5): -1,
 datetime.date(2022, 7, 7): -1,
 datetime.date(2022, 7, 8): -1,
 datetime.date(2022, 7, 10): 1,
 datetime.date(2022, 7, 13): -1,
 datetime.date(2022, 7, 18): 0,
 datetime.date(2022, 7, 27): -1,
 datetime.date(2022, 7, 29): 1,
 datetime.date(2022, 8, 2): -1,
 datetime.date(2022, 8, 14): -1,
 datetime.date(2022, 8, 19): 

In [120]:
tmp = daily_votes

In [121]:
tmp

{datetime.date(2022, 6, 1): -1,
 datetime.date(2022, 6, 2): -1,
 datetime.date(2022, 6, 6): -1,
 datetime.date(2022, 6, 7): 1,
 datetime.date(2022, 6, 8): -1,
 datetime.date(2022, 6, 9): -1,
 datetime.date(2022, 6, 10): -1,
 datetime.date(2022, 6, 15): -1,
 datetime.date(2022, 6, 17): -1,
 datetime.date(2022, 6, 18): -1,
 datetime.date(2022, 6, 20): 0,
 datetime.date(2022, 6, 22): -1,
 datetime.date(2022, 6, 23): -1,
 datetime.date(2022, 6, 24): -1,
 datetime.date(2022, 6, 25): -1,
 datetime.date(2022, 6, 27): -1,
 datetime.date(2022, 7, 1): -1,
 datetime.date(2022, 7, 2): -1,
 datetime.date(2022, 7, 3): -1,
 datetime.date(2022, 7, 4): -1,
 datetime.date(2022, 7, 5): -1,
 datetime.date(2022, 7, 7): -1,
 datetime.date(2022, 7, 8): -1,
 datetime.date(2022, 7, 10): 1,
 datetime.date(2022, 7, 13): -1,
 datetime.date(2022, 7, 18): 0,
 datetime.date(2022, 7, 27): -1,
 datetime.date(2022, 7, 29): 1,
 datetime.date(2022, 8, 2): -1,
 datetime.date(2022, 8, 14): -1,
 datetime.date(2022, 8, 19): 

In [122]:
daily_votes = tmp

In [123]:
trade_days = sorted(transaction_df.loc[(transaction_df['stock_name']=='聯發科')]['date'])
#turn elements in trade_days to date
trade_days = pd.to_datetime(trade_days).date
trade_days = trade_days[62:]

#if a date it not in daily_votes, add it to daily_votes and set its value to 0
for date in trade_days:
    if date not in daily_votes:
        daily_votes[date] = 0

#if a date is in daily_votes but not in trade_days, remove it from daily_votes
for date in list(daily_votes.keys()):
    if date not in trade_days:
        daily_votes.pop(date)
#trade_days = [datetime.datetime.strptime(date[:10], "%Y-%m-%d") for date in trade_days]
#trade_days

In [124]:
trade_days[63]

datetime.date(2022, 8, 29)

In [125]:
vote_list = [(key, value) for key, value in daily_votes.items()]

In [126]:
#change first column of vote_list to string from datetime
vote_list = [(str(key), value) for key, value in daily_votes.items()]

In [127]:
#sort vote_list by date
vote_list = sorted(vote_list, key = lambda x: x[0])
vote_list


[('2022-05-31', 0),
 ('2022-06-01', -1),
 ('2022-06-02', -1),
 ('2022-06-06', -1),
 ('2022-06-07', 1),
 ('2022-06-08', -1),
 ('2022-06-09', -1),
 ('2022-06-10', -1),
 ('2022-06-13', 0),
 ('2022-06-14', 0),
 ('2022-06-15', -1),
 ('2022-06-16', 0),
 ('2022-06-17', -1),
 ('2022-06-20', 0),
 ('2022-06-21', 0),
 ('2022-06-22', -1),
 ('2022-06-23', -1),
 ('2022-06-24', -1),
 ('2022-06-27', -1),
 ('2022-06-28', 0),
 ('2022-06-29', 0),
 ('2022-06-30', 0),
 ('2022-07-01', -1),
 ('2022-07-04', -1),
 ('2022-07-05', -1),
 ('2022-07-06', 0),
 ('2022-07-07', -1),
 ('2022-07-08', -1),
 ('2022-07-11', 0),
 ('2022-07-12', 0),
 ('2022-07-13', -1),
 ('2022-07-14', 0),
 ('2022-07-15', 0),
 ('2022-07-18', 0),
 ('2022-07-19', 0),
 ('2022-07-20', 0),
 ('2022-07-21', 0),
 ('2022-07-22', 0),
 ('2022-07-25', 0),
 ('2022-07-26', 0),
 ('2022-07-27', -1),
 ('2022-07-28', 0),
 ('2022-07-29', 1),
 ('2022-08-01', 0),
 ('2022-08-02', -1),
 ('2022-08-03', 0),
 ('2022-08-04', 0),
 ('2022-08-05', 0),
 ('2022-08-08', 0),


In [128]:
#save the vote_list as csv
with open('Prediction_dcard.csv', 'w') as file:
    for key, value in vote_list:
        file.write('%s,%s\n' % (key, value))